## Building A Chatbot
We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. 

In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "llama-3.3-70b-versatile")

In [5]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hello my name is Safeen Khan and I work at Accenture")])

AIMessage(content="Hello Safeen Khan, nice to meet you. It's great that you work at Accenture, a leading global professional services company. What do you do at Accenture, if you don't mind me asking? Are you working in a specific department or project?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 48, 'total_tokens': 102, 'completion_time': 0.149819304, 'prompt_time': 0.002193019, 'queue_time': 0.0561167, 'total_time': 0.152012323}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4181589c-4195-43db-9c29-2a1e1743249e-0', usage_metadata={'input_tokens': 48, 'output_tokens': 54, 'total_tokens': 102})

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hello my name is Safeen Khan and I work at Accenture"),
        AIMessage(content = "Hello Safeen Khan! It's nice to meet you. Accenture is a well-known and respected company, so that's great that you're a part of their team. What do you do at Accenture, if you don't mind me asking? Are you working on any exciting projects or initiatives?"),
        HumanMessage(content = "What is my name and where do I work")
    ]
)

AIMessage(content='Your name is Safeen Khan, and you work at Accenture.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 128, 'total_tokens': 143, 'completion_time': 0.032296024, 'prompt_time': 0.006322502, 'queue_time': 0.056734068, 'total_time': 0.038618526}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c374e504-f69c-4bbf-8f33-5d6b72529ff6-0', usage_metadata={'input_tokens': 128, 'output_tokens': 15, 'total_tokens': 143})

We can see the model has some history and can remember the content that we have told

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id : str) ->BaseChatMessageHistory:    #BaseChatMessageHistory is the return type
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [8]:
config = {"configurable":{
    "session_id" : "chat1"
}}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello my name is Safeen Khan and I work at Accenture")],
    config = config)

In [10]:
response.content

"Hello Safeen Khan, nice to meet you. It's great that you work at Accenture, a renowned global consulting firm. What do you do at Accenture, if I may ask? Are you part of a specific practice or team, such as technology, strategy, or operations? I'm here to help and chat, so feel free to share more about your work or anything else you'd like to discuss."

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="what is my name ?")],
    config = config)
response.content

'Your name is Safeen Khan.'

Now if we change the config or main the session_id we will se it doen't remembers my name

In [12]:
config2 = {"configurable":{
    "session_id":"chat2"
}}

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="what is my name ?")],
    config = config2
)
response.content

"I don't have any information about your name. I'm a large language model, I don't have personal interactions or memories, and I don't retain any information about individual users. Each time you interact with me, it's a new conversation. If you'd like to share your name with me, I'd be happy to chat with you!"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [17]:
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are the world best system design engineer answer my question accordingly"),
        MessagesPlaceholder(variable_name='message')
    ]
)
chain = prompt | model

In [18]:
chain.invoke({
    'message':[HumanMessage(content="What is pub-sub model , explain with example")]
})

AIMessage(content='The pub-sub model, short for Publish-Subscribe model, is a design pattern that allows for the decoupling of producers and consumers of data. In this model, publishers (or producers) send messages to a topic or channel, and subscribers (or consumers) listen to these topics or channels to receive the messages.\n\nHere\'s a breakdown of the pub-sub model:\n\n**Components:**\n\n1. **Publisher** (or Producer): The entity that sends messages to a topic or channel.\n2. **Subscriber** (or Consumer): The entity that listens to a topic or channel to receive messages.\n3. **Topic** (or Channel): The medium through which messages are sent and received.\n4. **Broker** (or Message Queue): The intermediary that manages the topics or channels and routes messages from publishers to subscribers.\n\n**How it works:**\n\n1. A publisher sends a message to a topic or channel.\n2. The broker receives the message and stores it in the topic or channel.\n3. Subscribers listen to the topic or 

In [19]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)
config = {'configurable':{'session_id':'chat3'}}
response = with_message_history.invoke(
    [HumanMessage(content='In pub sub model how is data transfered , like if we are using aws sns , they just send a message on topic , but what if we want some data to be transfered when that subscriber is called')],
    config = config
)
response

AIMessage(content='**Data Transfer in Pub/Sub Model**\n\nIn the Pub/Sub model, data transfer occurs when a subscriber receives a notification from the topic. The notification typically includes a message or payload that contains the data to be transferred. Here\'s a step-by-step explanation of how data transfer works in the Pub/Sub model:\n\n1. **Publisher sends a message**: The publisher sends a message to the topic, which includes the data to be transferred.\n2. **Message is stored in the topic**: The message is stored in the topic, and the topic notifies the subscribers that a new message is available.\n3. **Subscriber receives notification**: The subscriber receives a notification that a new message is available in the topic.\n4. **Subscriber requests the message**: The subscriber sends a request to the topic to retrieve the message.\n5. **Message is delivered to the subscriber**: The topic delivers the message to the subscriber, which includes the data to be transferred.\n\n**Usin

Adding more complexity to the model by making two variable as dynamic

In [38]:
dynamicPrompt = ChatPromptTemplate.from_messages([
    ('system',"You are a world class language translator , translate the following sentense in {language}"),
    MessagesPlaceholder(variable_name='messages')
])
chain2 = dynamicPrompt | model

In [40]:
with_message_history = RunnableWithMessageHistory(chain2,get_session_history,input_messages_key = 'messages')
config = {'configurable':{'session_id':'chat4'}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="My name is Mohd Safeen Khan , I work as Gen AI engineer at Accenture")],"language":"Hindi"},
    config=config
)
response

AIMessage(content='मेरा नाम मोहम्मद सफीन खान है, मैं एक्सेंचर में जनरल एआई इंजीनियर के रूप में काम करता हूँ।', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 105, 'total_tokens': 152, 'completion_time': 0.136284942, 'prompt_time': 0.005207611, 'queue_time': 0.052621189, 'total_time': 0.141492553}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--aa7b9051-89e2-4c42-807e-b3f920b3c908-0', usage_metadata={'input_tokens': 105, 'output_tokens': 47, 'total_tokens': 152})